# Variables climáticas anuales a partir de climáticas diarias

Este archivo es importante para pasar de 19 filas (19 distritos) del df_0 a 19 (ditritos) x 13 (años) = 247 filas
Ya tenemos datos de incendios por distrito entre el 2007 y el 2019 incluidos
 En este df_1 asumimos que la superficie se mantiene constante.
 Naturalmente, al darle más variabilidad climática le dará más peso al clima
 y menos a las variables específicas. Aún así es interesante

 Tenemos descargados todos los csv
 para todos los distritos, pluviosidad diaria entre el 2007 y el 2019
 Excepto para el distrito 5 que tenemos desde el 2010 al 2019
 los archivos están en la carpeta "climati_csv" alojada en la misma que el codigo
 Los csv están nombrados de forma sistemática: distrit01_plu_ju07-no11: # pluviosidad en distrito 1 entre el 10 junio del 2007 y el 10 noviembre del 2011
 A ver que se hace con el distrito 5: imputar o no usar los años faltantes



In [12]:
import pandas as pd
import os

DATOS_DIR = os.path.join(os.getcwd(), 'climati_csv')

def load_dataframe(file):
    distrito = int(file[7:9])
    df = pd.read_csv(os.path.join(DATOS_DIR, file))
    df['distrito'] = distrito
    return df

df = pd.concat([
    load_dataframe(x) for x in os.listdir(DATOS_DIR)
])

In [14]:
df.sample(10)

,Instante lectura,Chuvia,distrito
665,2019-04-06 00:00:00.0,27.0,18
325,2018-05-01 00:00:00.0,1.6,11
191,2012-12-18 00:00:00.0,0.8,15
28,2010-07-08 00:00:00.0,0.2,5
1502,2016-07-21 00:00:00.0,0.0,15
833,2014-09-21 00:00:00.0,0.0,1
253,2018-02-18 00:00:00.0,0.0,18
1392,2016-04-02 00:00:00.0,9.8,17
370,2008-06-14 00:00:00.0,0.0,17
1039,2015-04-15 00:00:00.0,1.8,1


In [6]:
df_dis1_1 = pd.read_csv("climati_csv/distrit01_plu_ju07-no11.csv")


In [10]:
df_dis1_1.describe(include='all')

,Instante lectura,Chuvia
count,1615,1615.000000
unique,1615,NaN
top,2007-07-13 00:00:00.0,NaN
freq,1,NaN
mean,NaN,2.919814
std,NaN,5.884686
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,3.050000


In [8]:
# Ver si hay huecos entre las fechas

df_dis1_1['Instante lectura'] = pd.to_datetime(df_dis1_1['Instante lectura'] )
from datetime import timedelta
test = sum(~(df_dis1_1['Instante lectura'] - df_dis1_1['Instante lectura'].shift(1) == timedelta(days=1)))
print("Hay {} huecos".format(test-1) if test > 1 else "No hay huecos")

No hay huecos


In [ ]:
# Lo cual es lógico, ya que descargué del 10 de jun 2007 al 10 de nov 2011
# Estaría bien borrar todos los días entre el 10 de nov y el 10 de jun, 
# así igualamos datos entre años 2007 y 2011 con años 2008, 2009 y 2010

In [9]:
# Descargamos a mano los otros dos de ferrol

df_dis1_2 = pd.read_csv("climati_csv/distrit01_plu_ju12-no16.csv")
df_dis1_2.head(5)

,Instante lectura,Chuvia
0,2012-06-10 00:00:00.0,5.2
1,2012-06-11 00:00:00.0,7.9
2,2012-06-12 00:00:00.0,2.6
3,2012-06-13 00:00:00.0,0.0
4,2012-06-14 00:00:00.0,1.9


In [10]:
df_dis1_3 = pd.read_csv("climati_csv/distrit01_plu_ju17-no19.csv")
df_dis1_3.head(5)

,Instante lectura,Chuvia
0,2017-06-10 00:00:00.0,0.0
1,2017-06-11 00:00:00.0,0.0
2,2017-06-12 00:00:00.0,0.0
3,2017-06-13 00:00:00.0,0.0
4,2017-06-14 00:00:00.0,0.0


In [12]:
# Antes que nada, unir los tres. 
# De este mondo el codigo se aplicará sobre solo un df
# ¿Valdrá lo siguiente?:

# A partir de df_dis1_1, df_dis1_2, df_dis1_3:

df_distrit01 = df_dis1_1.append(df_dis1_2)
df_distrit01 = df_distrit01.append(df_dis1_3)

df_distrit01

,Instante lectura,Chuvia
0,2007-06-10 00:00:00,0.2
1,2007-06-11 00:00:00,0.2
2,2007-06-12 00:00:00,2.0
3,2007-06-13 00:00:00,13.0
4,2007-06-14 00:00:00,5.0
...,...,...
879,2019-11-06 00:00:00.0,16.0
880,2019-11-07 00:00:00.0,8.3
881,2019-11-08 00:00:00.0,14.7
882,2019-11-09 00:00:00.0,13.7


In [14]:
# Con este código nos quedamos para cada año con los días entre
# el 10 de junio y el 10 de noviembre

df_distrit01['Instante lectura'] = pd.to_datetime(df_distrit01['Instante lectura'])
df_meses_completos = df_distrit01[df_distrit01['Instante lectura'].dt.month.isin([7, 8, 9, 10])]
df_junio = df_distrit01[(df_distrit01['Instante lectura'].dt.month == 6) & (df_distrit01['Instante lectura'].dt.day >= 10)]
df_noviembre = df_distrit01[(df_distrit01['Instante lectura'].dt.month == 11) & (df_distrit01['Instante lectura'].dt.day <= 10)]
df = pd.concat([df_meses_completos, df_junio, df_noviembre])
df

# ver pandas y series temporales para atribuir
# frecuencas a intervalos de dias seguidos sin llover
# formula de excel para columna de acumulados: =SI(B2=0;SI(B1=0;C1+1;1);0)

# Antes que nada eliminamos filas innecesarias

# Habrá que hacer un drop if date in (0-11,31-12) y (1-1,10-6) 

,Instante lectura,Chuvia
21,2007-07-01,0.6
22,2007-07-02,0.2
23,2007-07-03,8.2
24,2007-07-04,0.0
25,2007-07-05,0.0
...,...,...
879,2019-11-06,16.0
880,2019-11-07,8.3
881,2019-11-08,14.7
882,2019-11-09,13.7


In [ ]:
# son registros de 13 años, a 152 días por año (entre 10 de junio y 10 de noviembre)
# 13 x 154 = 2002. Todo ok 

# ¿Que datos anuales necesitaré?
# Idea:
# Lluvia total
# Numero de días sin lluvia
# Numero máximo de días seguidos sin lluvia
# Número de veces que hubo 7 días sin llover
# Número de veces que hubo 15 días días sin llover

# Otra idea:
# Obtener valores anuales para cada distrito de días seguidos sin llover
# entre 6 y 13, entre 14 y 21, entre 22 y 29, 30 y 37, y 38 o más 
# contando cada día sin llover solo en uno de los intervalos a ser posible
# Luego podremos ir probando con qué columnas funciona mejor el algoritmo

# En cualquier caso, habrá que hacer un groupby anual ....




In [ ]:
# Hay que pensar como queremos que nos queden los datos
# El resto de las variables están en: TFM_incendios_galicia_1.ipynb

In [ ]:
# Veamos qué pasa en un solo csv, que meteremos en un dfp (prueba)

dfp = pd.read_csv("climati_csv/distrit01_plu_ju07-no11.csv")
dfp.describe(include='all')


# Vemos que hay 1615 lecturas. demasiadas para 5 medios años...
# Descargamos a mano los otros dos de ferrol

df_dis1_2 = pd.read_csv("climati_csv/distrit01_plu_ju12-no16.csv")
df_dis1_3 = pd.read_csv("climati_csv/distrit01_plu_ju17-no19.csv")


dfp = dfp.append(df_dis1_2)
dfp = dfp.append(df_dis1_3)

# Nos quedamos solo con los días entre el 10 de junio y 10 de noviembre incluidos
# primero pasamos a formato fecha el instante de lectura

dfp['Instante lectura'] = pd.to_datetime(dfp['Instante lectura'])

df_meses_completos = dfp[dfp['Instante lectura'].dt.month.isin([7, 8, 9, 10])]
df_junio = dfp[(dfp['Instante lectura'].dt.month == 6) & (dfp['Instante lectura'].dt.day >= 10)]
df_noviembre = dfp[(dfp['Instante lectura'].dt.month == 11) & (dfp['Instante lectura'].dt.day <= 10)]
dfp = pd.concat([df_junio, df_meses_completos, df_noviembre])

dfp.shape


Vemos que 13 años x 154 dias = 2002 días registrados en el distrito 1

Es decir, este distrito está completo. En el resto de los distritos en total hay 11 huecos, que habrá que imputar. En el caso del distrito 5 se podrán obviar o copiar del distrito 4. Después de todo no hay mucha diferencia de clima entre costa da morte(distrito 5) y el barbanza (distrito 4)

De todas formas, seguimos sin comprender muy bien dónde están esas 324 filas que faltan...

¿Revisaremos uno a uno todos los distritos? 

¿Es posible crear un bucle para hacerlo?

otro acercamiento: ver si hay huecos.

In [ ]:
# Intentaremos ver dónde hay huecos. Lo que hacemos es subir la fecha un día y restarlo al original.
# Si la diferencia es 1 no habrá huecos en ese día

from datetime import timedelta

df['year'] = df['Instante lectura'].dt.year

def groupby_diff(inner_df):
    test = sum(~(inner_df['Instante lectura'] - inner_df['Instante lectura'].shift(1) == timedelta(days=1))) - 1
    return test

huecos = df.groupby(by=['distrito', 'year']).apply(groupby_diff)
huecos[huecos > 0]

# Lo que estamos haciendo aquí es restar las fechas originales a fechas atrasadas
# Si hay huecos en las fechas este valor es distinto de 1
# Conceptos avanzados: grooupbydiff, inner_df, timedelta.

In [ ]:
# K-FOLD NEIGHBORS REGRESSOR

def Round2(X, y):
    # Set parameters
    min_score = {}
    for neigh in [5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000]:

        model = KNeighborsRegressor(n_neighbors=neigh)
        n = len(y)

        # Perform 5-fold cross validation
        scores = []
        kf = KFold(n, n_folds=5, shuffle=True)

        # Calculate mean absolute deviation for train/test for each fold
        for train_idx, test_idx in kf:
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, prediction))
            # score = model.score(X_test, y_test)
            scores.append(rmse)
        if len(min_score) == 0:
            min_score['neighbor'] = neigh
            min_score['scores'] = scores
        else:
            if np.mean(scores) < np.mean(min_score['scores']):
                min_score['neighbor'] = neigh
                min_score['scores'] = scores
        print "Neighbors:", neigh
        print scores
        print np.mean(scores)
    return min_score
